### Real World application 3 (Chapitre 4)
create database pubmed 

use pubmed 

In [17]:
!pip install py2neo

In [34]:
import csv
import sys
from py2neo import Graph, Node

# Augmenter la taille maximale du champ
csv.field_size_limit(sys.maxsize)

# Connexion à Neo4j
graph = Graph("bolt://localhost:7687", auth=("neo4j", "password_tuts"), name="pubmed")

# Ouvrir et lire le fichier CSV
with open("pubmed.csv", "r", newline="", encoding="utf-8") as csvfile:
    reader = csv.DictReader(csvfile)
    
    # Parcourir chaque ligne du CSV
    for row in reader:
        # Extraire les informations pertinentes de chaque ligne
        pmid = row["pmid"]
        title = row["title"]
        authors = row["authors"]
        
        # Splitter la chaîne des auteurs pour obtenir une liste d'auteurs
        authors_list = authors.split("\n")
        
        # Parcourir chaque auteur
        for author_str in authors_list:
            # Extraire le nom et l'affiliation de l'auteur
            author_info = author_str.split(", affil str ")
            name = author_info[0].strip()
            
            # Vérifier si author_info contient suffisamment d'éléments
            if len(author_info) > 1:
                affiliation = author_info[1].strip()
            else:
                affiliation = "Unknown"  # Ou toute autre valeur par défaut
            
            # Créer un nœud pour l'auteur s'il n'existe pas déjà
            author_node = Node("Author", name=name, affiliation=affiliation)
            graph.create(author_node)
            
        # Créer des relations entre les auteurs qui ont collaboré sur le même article
        for i in range(len(authors_list)):
            for j in range(i+1, len(authors_list)):
                author1 = authors_list[i].split(", affil str ")[0].strip()
                author2 = authors_list[j].split(", affil str ")[0].strip()
                query = """
                    MATCH (a1:Author {name: $author1}), (a2:Author {name: $author2})
                    MERGE (a1)-[:COLLABORATED_WITH]->(a2)
                """
                graph.run(query, author1=author1, author2=author2)


In [ ]:
# Question 1: Combien d'auteurs ont plus de 2 affiliations ?
query_q1 = """
    MATCH (a:Author)
    WHERE size(a.affiliations) > 2
    RETURN count(a) AS totalAuthors
"""
result_q1 = graph.run(query_q1).data()

# Afficher le nombre d'auteurs ayant plus de 2 affiliations
total_authors_more_than_2_affiliations = result_q1[0]['totalAuthors']
print("Nombre d'auteurs ayant plus de 2 affiliations :", total_authors_more_than_2_affiliations)

In [ ]:
# Question 2: Créez une densité de degrés de nœuds. Observez-vous une loi d'échelle de puissance ?

# Calculer la densité de degrés ici et analyser si une loi d'échelle de puissance est observée

# Calcul de la densité des degrés des nœuds
query_q2 = """
    CALL algo.degree.stream()
    YIELD nodeId, score
    RETURN score AS degree
"""
result_q2 = graph.run(query_q2).data()

# Collecter les degrés dans une liste
degrees = [record["degree"] for record in result_q2]

# Analyse de la loi d'échelle de puissance
# Vous pouvez utiliser des bibliothèques telles que numpy, scipy ou powerlaw pour effectuer cette analyse

# Par exemple, avec numpy
import numpy as np

# Calculer la moyenne et l'écart-type des degrés
mean_degree = np.mean(degrees)
std_degree = np.std(degrees)

# Afficher la moyenne et l'écart-type
print("Moyenne des degrés:", mean_degree)
print("Écart-type des degrés:", std_degree)

# Visualiser la distribution des degrés
import matplotlib.pyplot as plt

plt.hist(degrees, bins=50, density=True, alpha=0.6, color='g')
plt.xlabel('Degré des nœuds')
plt.ylabel('Densité')
plt.title('Distribution des degrés des nœuds')
plt.show()

In [ ]:
# Question 3: Quel pays a connu le plus de collaborations internationales ?

# Query pour identifier les collaborations internationales
query_q3 = """
    MATCH (a:Author)
    WHERE size(a.affiliations) > 1
    RETURN a.name, a.affiliations
"""
result_q3 = graph.run(query_q3).data()

# Analyser les affiliations pour déterminer les collaborations internationales et les pays impliqués
collaborations_by_country = {}

for record in result_q3:
    author_name = record["a.name"]
    affiliations = record["a.affiliations"]
    
    # Examinez chaque affiliation pour détecter les pays impliqués
    for affiliation in affiliations:
        country = affiliation.split(", ")[-1]  # Supposons que le dernier élément de l'affiliation est le pays
        if country not in collaborations_by_country:
            collaborations_by_country[country] = 0
        collaborations_by_country[country] += 1

# Trouver le pays avec le plus de collaborations internationales
most_collaborative_country = max(collaborations_by_country, key=collaborations_by_country.get)
total_collaborations = collaborations_by_country[most_collaborative_country]

print("Le pays avec le plus de collaborations internationales est :", most_collaborative_country)
print("Nombre total de collaborations internationales :", total_collaborations)


In [ ]:
# Question 4: Qui sont les 10 meilleurs collaborateurs et quelle est leur affiliation ?
query_q4 = """
    MATCH (a:Author)-[:COLLABORATED_WITH]->(b:Author)
    RETURN a.name, COUNT(b) AS collaborations, a.affiliations
    ORDER BY collaborations DESC
    LIMIT 10
"""
result_q4 = graph.run(query_q4).data()
for idx, row in enumerate(result_q4, start=1):
    print(f"{idx}. {row['a.name']} - {row['a.affiliations']} (Collaborations: {row['collaborations']})")

In [ ]:
# Exercice sandbox : Trouver les auteurs qui sont des ponts entre différents domaines de recherche

# Définir une fonction pour calculer le nombre d'affiliations uniques avec lesquelles un auteur a collaboré
def unique_affiliations_count(author_name):
    query = """
        MATCH (a:Author {name: $author_name})-[:COLLABORATED_WITH]->(b:Author)
        RETURN DISTINCT b.affiliations AS affiliations
    """
    results = graph.run(query, author_name=author_name).data()
    unique_affiliations = set()
    for result in results:
        for affiliation in result["affiliations"]:
            unique_affiliations.add(affiliation)
    return len(unique_affiliations)

# Trouver les auteurs qui sont des ponts entre différents domaines de recherche
query_sandbox = """
    MATCH (a:Author)
    RETURN a.name, a.affiliations
"""
results_sandbox = graph.run(query_sandbox).data()

bridge_authors = []
for result in results_sandbox:
    author_name = result["a.name"]
    affiliations_count = unique_affiliations_count(author_name)
    if affiliations_count > 1:  # Si l'auteur a collaboré avec des affiliations de plus d'un domaine
        bridge_authors.append({"name": author_name, "affiliations_count": affiliations_count})

# Afficher les auteurs ponts avec le nombre d'affiliations de domaines différents avec lesquelles ils ont collaboré
print("Auteurs ponts entre différents domaines de recherche:")
for idx, author in enumerate(bridge_authors, start=1):
    print(f"{idx}. {author['name']} - Nombre d'affiliations uniques: {author['affiliations_count']}")